# Real Book API

This is where I'll test how best to split up classes. As an overview, I'll need to implement:

- Song
  - Title
  - Key
  - Time Signature
  - Tempo
  - Measures
    - Notes
    - Chords
    - Lyrics

Ideally, the API will be able to display the song as a PDF or, if using the CLI, display the song in the terminal.

## Implementation

Let's deal with our imports first.

In [ ]:
import math
import numbers

from decimal import Decimal
from enum import Enum, EnumType
from fractions import Fraction
from pydantic import BaseModel
from typing import Annotated, Any, Dict, List, Literal, Tuple, Union

Let's create the lower level objects first:

- Notes
- Chords
- Lyrics

First, let's create the constant, acceptable items available for notes and chords.

In [9]:
class Notes(str, Enum):
  A_FLAT = "Ab"
  A_NATURAL = "A"
  A_SHARP = "A#"
  B_FLAT = "Bb"
  B_NATURAL = "B"
  C_FLAT = "Cb"
  C_NATURAL = "C"
  C_SHARP = "C#"
  D_FLAT = "Db"
  D_NATURAL = "D"
  D_SHARP = "D#"
  E_FLAT = "Eb"
  E_NATURAL = "E"
  F_FLAT = "Fb"
  F_NATURAL = "F"
  F_SHARP = "F#"
  G_FLAT = "Gb"
  G_NATURAL = "G"
  G_SHARP = "G#"

In [10]:
class Chords(str, Enum):
  MAJOR = "Maj"
  MINOR = "Min"
  AUGMENTED = "Aug"
  DIMINISHED = "Dim"
  SUSPENDED_TWO = "Sus2"
  SUSPENDED_FOUR = "Sus4"
  SEVEN = "7"

In [11]:
class Qualities(str, Enum):
  FIVE_FLAT = "b5"
  FIVE_SHARP = "#5"
  SIX_FLAT = "b6"
  SIX = "6"
  SEVEN_FLAT = "b7"
  SEVEN = "7"
  NINE_FLAT = "b9"
  NINE = "9"
  NINE_SHARP = "#9"
  ELEVEN = "11"
  ELEVEN_SHARP = "#11"
  THIREEN_FLAT = "b13"
  THIRTEEN = "13"

Can't forget about note lengths. Because we'll run into problems with fractions and Enums, we need to create a FractionEnumMeta.

In [ ]:
class FractionEnumMeta(type(Fraction), EnumType):
  pass


class FractionEnum(Fraction, Enum, metaclass=FractionEnumMeta):
  def __new__(cls, numerator=0, denominator=None):
    self = object.__new__(cls)

    if denominator is None:
      if type(numerator) is int:
        self._numerator = numerator
        self._denominator = 1
        return self
      elif isinstance(numerator, numbers.Rational):
        self._numerator = numerator.numerator
        self._denominator = numerator.denominator
        return self
      elif isinstance(numerator, (float, Decimal)):
        self._numerator, self._denominator = numerator.as_integer_ratio()
        return self
      elif isinstance(numerator, str):
        m = _RATIONAL_FORMAT.match(numerator)

In [ ]:
class Length(Fraction, Enum):
  WHOLE = Fraction(1, 1)
  # WHOLE_TRIPLET = 0.33
  # HALF = 0.5
  # HALF_DOTTED = 0.75
  # QUARTER = 0.25
  # QUARTER_DOTTED = 0.375
  # EIGHTH = 0.125
  # EIGHTH_DOTTED = 0.3125
  # SIXTEENTH = 0.0625
  # SIXTEENTH_DOTTED = 0.09375
  # THIRTYSECOND = 0.03125
  # THIRTYSECOND_DOTTED = 0.046875